In [16]:
import os
import pandas as pd
import re
import copy

cur_dir = os.getcwd()

folder_list = ["My_BF_deque_all_ntc","My_BF_queue_all_ntc","My_FW_a_all_ntc","My_FW_g_all_ntc","My_spdial_all_ntc","My_spheap_all_ntc"]



def find_index_first(list_to_search,search_text):
    for i, string in enumerate(list_to_search):
        if len(string) > len(search_text):
            if string[:len(search_text)] == search_text:
                return i
    return -1   # Return -1 if no string found with first letter 'n'


summary_dict = dict()
score = {"time" : 0,"distance" : 0, "non-trival triple comparison" : 0}


for folder in folder_list:
    fld_dir = os.path.join(cur_dir,folder)
    for dirPath, dirNames, fileNames in os.walk(fld_dir):
        for f in fileNames:
            if (f[-4:] == ".txt") :
                split_list = re.split(r'[_.]', f)
                # print(split_list)
                
                
                x_idx = find_index_first(split_list,"x")
                y_idx = find_index_first(split_list,"y")
                seed_idx = find_index_first(split_list,"seed")
                method_list = split_list[:x_idx]
                cond_list = split_list[x_idx:seed_idx+1]
                method = "_".join(method_list)
                cond = "_".join(cond_list)

                # print(method)
                # print(cond)

                if cond not in summary_dict:
                    summary_dict[cond] = dict()
                if method not in summary_dict[cond]:
                    summary_dict[cond][method] = copy.deepcopy(score)

                file_path = os.path.join(dirPath, f)
                file = open(file_path, 'r')
                text = []
                distance_cum = 0
                non_triple_comparison = 0
                for line in file:
                    text.append(line)
                    if line[:4] == "Node":
                        split_list = re.split('; |: ', line)
                        if(len(split_list) == 4):
                            distance = int(split_list[-3])
                            ntc = int(split_list[-1])
                            distance_cum += distance
                            non_triple_comparison += ntc
                        elif(len(split_list) == 2):
                            distance = int(split_list[-1])
                            distance_cum += distance
                    if line[:3] == "non":
                        ntc = int(re.search(r': (\d+)', line).group(1))
                        non_triple_comparison += ntc
                    if line[:3] == "Run":
                        n_it = int(re.search(r'(\d+) iteration', line).group(1))
                        time = float(re.search(r': (\d+\.\d+)', line).group(1))
                        summary_dict[cond][method]["time"] = time/n_it
                        summary_dict[cond][method]["distance"] = distance_cum/n_it
                        summary_dict[cond][method]["non-trival triple comparison"] = non_triple_comparison/n_it
                file.close

print(summary_dict)

{'x10_y10_c1min1_c1max1000_c2min1_c2max1000_rmin0_1_rmax1_seed2': {'My_BF_deque_all_ntc': {'time': 0.5, 'distance': 3786860.0, 'non-trival triple comparison': 21344.0}, 'My_BF_queue_all_ntc': {'time': 0.6, 'distance': 3786860.0, 'non-trival triple comparison': 18383.0}, 'My_FW_a_all_ntc': {'time': 0.2, 'distance': 3786860.0, 'non-trival triple comparison': 17192.0}, 'My_FW_g_all_ntc': {'time': 0.5, 'distance': 3786860.0, 'non-trival triple comparison': 17192.0}, 'My_spdial_all_ntc': {'time': 1.4, 'distance': 3786860.0, 'non-trival triple comparison': 13729.0}, 'My_spheap_all_ntc': {'time': 0.6, 'distance': 3786860.0, 'non-trival triple comparison': 13729.0}}, 'x100_y50_c1min1_c1max1000_c2min1000_c2max10000_rmin0_1_rmax1_seed20': {'My_BF_deque_all_ntc': {'time': 2292.0, 'distance': 23222300700.0, 'non-trival triple comparison': 220854880.0}, 'My_BF_queue_all_ntc': {'time': 524.0, 'distance': 23222300700.0, 'non-trival triple comparison': 39524046.0}, 'My_FW_a_all_ntc': {'time': 25694.0,

In [19]:
df = pd.DataFrame.from_dict({(i,j): summary_dict[i][j] 
                             for i in summary_dict.keys() 
                             for j in summary_dict[i].keys()},
                            orient='index')
df[["distance","non-trival triple comparison"]] = df[["distance","non-trival triple comparison"]].astype(int)
display(df.head(30))
final_summary_path = os.path.join(cur_dir,"summary_table.xlsx")
df.to_excel(final_summary_path)

time  \
x10_y10_c1min1_c1max1000_c2min1_c2max1000_rmin0... My_BF_deque_all_ntc      0.5   
                                                   My_BF_queue_all_ntc      0.6   
                                                   My_FW_a_all_ntc          0.2   
                                                   My_FW_g_all_ntc          0.5   
                                                   My_spdial_all_ntc        1.4   
                                                   My_spheap_all_ntc        0.6   
x100_y50_c1min1_c1max1000_c2min1000_c2max10000_... My_BF_deque_all_ntc   2292.0   
                                                   My_BF_queue_all_ntc    524.0   
                                                   My_FW_a_all_ntc      25694.0   
                                                   My_FW_g_all_ntc      25337.0   
                                                   My_spdial_all_ntc     1060.0   
                                                   My_spheap_all_ntc     1793.0   
x50_y25_c1min1000_c1max10000_c2min1_c2max1000_r... My_BF_deque_all_ntc     39.0   
                                                   My_BF_queue_all_ntc     44.0   
                                                   My_FW_a_all_ntc        398.0   
                                                   My_FW_g_all_ntc        385.0   
                                                   My_spdial_all_ntc      133.0   
                                                   My_spheap_all_ntc       70.0   
x50_y10_c1min1_c1max1000_c2min1000_c2max10000_r... My_BF_deque_all_ntc      5.0   
                                                   My_BF_queue_all_ntc      4.0   
                                                   My_FW_a_all_ntc         26.0   
                                                   My_FW_g_all_ntc         31.0   
                                                   My_spdial_all_ntc       48.0   
                                                   My_spheap_all_ntc       15.0   
x10_y10_c1min1000_c1max10000_c2min1_c2max1000_r... My_BF_deque_all_ntc      0.1   
                                                   My_BF_queue_all_ntc      0.2   
                                                   My_FW_a_all_ntc          0.2   
                                                   My_FW_g_all_ntc          0.3   
                                                   My_spdial_all_ntc        7.7   
                                                   My_spheap_all_ntc        0.4   

                                                                           distance  \
x10_y10_c1min1_c1max1000_c2min1_c2max1000_rmin0... My_BF_deque_all_ntc      3786860   
                                                   My_BF_queue_all_ntc      3786860   
                                                   My_FW_a_all_ntc          3786860   
                                                   My_FW_g_all_ntc          3786860   
                                                   My_spdial_all_ntc        3786860   
                                                   My_spheap_all_ntc        3786860   
x100_y50_c1min1_c1max1000_c2min1000_c2max10000_... My_BF_deque_all_ntc  23222300700   
                                                   My_BF_queue_all_ntc  23222300700   
                                                   My_FW_a_all_ntc      23222300700   
                                                   My_FW_g_all_ntc      23222300700   
                                                   My_spdial_all_ntc    23222300700   
                                                   My_spheap_all_ntc    23222300700   
x50_y25_c1min1000_c1max10000_c2min1_c2max1000_r... My_BF_deque_all_ntc   6343105850   
                                                   My_BF_queue_all_ntc   6343105850   
                                                   My_FW_a_all_ntc       6343105850   
                                                   My_FW_g_all_ntc       6343105850   
                      